In [1]:
# coding: utf-8

import sys
sys.path.insert(0, '../../src')
import feedparser
import pandas as pd

import urllib

from postagem.Util import extract_domain, download_and_move_image, get_noticia_comercio
from lexical_analyzer_package import midia_lexical
from midia_postagem import midia_post
from Model.News import News
from Database import midia_table

from newsplease import NewsPlease
from bs4 import BeautifulSoup
import requests

import datetime

In [2]:
link = 'https://www1.folha.uol.com.br/ultimas-noticias/'

In [3]:
req = requests.get(link)

In [17]:
noticias = BeautifulSoup(req.text, "html.parser").find_all('div', class_='flex-cell')

In [21]:
for noticia in noticias:
    print(noticia.find_all('a', href=True)[0]['href'])
    article = NewsPlease.from_url(noticia.find_all('a', href=True)[0]['href'])
    row = {'titulos': [], 'links': [], 'noticia': [], 'image': [], 'abstract': [], 'date': []}
    if (article is not None):
        row['titulos'].append(article.title)
        row['noticia'].append(article.text)
        row['links'].append(article.url)
        row['abstract'].append(article.text)
        row['date'].append(article.date_publish)
        path_image = article.image_url
        if path_image == '' or path_image == None:
            row['image'].append(0)
        else:
            row['image'].append(download_and_move_image(article.image_url))
        news = News(row['abstract'], row['noticia'], row['date'], row['links'], row['titulos'], row['image'])
        try:
            print(row['titulos'])
            news_in_db = midia_table.check_news(news)
            print('news_in_db: ' + str(news_in_db))
            if (not news_in_db):
                row = pd.DataFrame(row)
                df, categories = midia_lexical.lexical_corpus_and_title(row)
                # DB categories
                if (categories != [set()]):
                    news.set_categories(categories)
                    midia_table.save_news(news)
                    midia_post.post_news(df)
        except:
            print('Empty News')

https://www1.folha.uol.com.br/mundo/2018/09/onu-lanca-estrategia-contra-escravidao-moderna-e-trafico-humano.shtml
['ONU lança estratégia contra escravidão moderna e tráfico humano']
news_in_db: False
 -- no categories -- 
https://www1.folha.uol.com.br/cotidiano/2018/09/tempo-para-travessia-em-semaforos-aumenta-em-12-vias-de-sao-paulo.shtml
['Tempo para travessia em semáforos aumenta em 12 vias de São Paulo']
news_in_db: False
 -- no categories -- 


In [11]:
noticias[0]

<div class="flex-cell"><div class="row "><div class="col col--xs-1-1 col--sm-1-1 col--md-1-1">
<!-- news-list -->
<ol class="u-list-unstyled">
<li class="c-main-headline c-main-headline--horizontal">
<div class="c-main-headline__media-wrapper">
<div class="c-masked-image c-masked-image--3x2">
<a class="c-main-headline__url" href="https://www1.folha.uol.com.br/mundo/2018/09/onu-lanca-estrategia-contra-escravidao-moderna-e-trafico-humano.shtml">
<img action="" alt="UN Secretary-General Guterres speaks during " at="" call="" class="c-main-headline__image" drug="" event="" global="" in="" nations="" new="" on="" problem"="" src="https://f.i.uol.com.br/fotografia/2018/09/24/15378217065ba94c0a7ce4b_1537821706_3x2_xs.jpg" the="" to="" united="" world="" york"=""/>
</a>
</div>
</div>
<div class="c-main-headline__wrapper">
<div class="c-main-headline__head">
<h3 class="c-headline__kicker c-kicker">
                    Mundo

            </h3>
<div class="c-modal-drop c-headline__action" data-mo